In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexPrograms

In [ ]:
graph = GraphOfConvexPrograms()

v1 = graph.add_vertex("v1")
x1 = v1.add_variable(2)
c1 = np.array([-3, 0])
v1.add_constraint(cp.norm_inf(x1 - c1) <= 1)

v2 = graph.add_vertex("v2")
x2 = v2.add_variable(2)
c2 = np.array([0, 2.5])
D2 = np.diag([.25, 2])
v2.add_constraint(cp.norm_inf(D2 @ (x2 - c2)) <= 1)

v3 = graph.add_vertex("v3")
x3 = v3.add_variable(2)
c3 = np.array([3, 0])
v3.add_constraint(cp.norm2(x3 - c3) <= 1)

v4 = graph.add_vertex("v4")
x4 = v4.add_variable(2)
c4 = np.array([0, -2.5])
D4 = np.diag([1, 2])
v4.add_constraint(cp.norm2(D4 @ (x4 - c4)) <= 1)

v5 = graph.add_vertex("v5")
x5 = v5.add_variable(2)
c5 = np.array([.3, .3])
v5.add_constraint(cp.norm1(x5 - c5) <= 1)

In [ ]:
for tail in graph.vertices:
    for head in graph.vertices:
        if tail != head:
            edge = graph.add_edge(tail, head)
            edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 2))

In [ ]:
graph.graphviz()

In [ ]:
prob = graph.solve_traveling_salesman()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
graph.plot_2d()
graph.plot_2d_subgraph()

# From ILP

In [ ]:
from itertools import combinations

ilp_constraints = []
yv = graph.vertex_binaries()
ye = graph.edge_binaries()

for i, v in enumerate(graph.vertices):
    inc_edges = graph.incoming_edge_indices(v)
    out_edges = graph.outgoing_edge_indices(v)
    ilp_constraints.append(yv[i] == 1)
    ilp_constraints.append(sum(ye[out_edges]) == 1)
    ilp_constraints.append(sum(ye[inc_edges]) == 1)

subtour_constraints = []
for r in range(2, graph.num_vertices() - 1):
    for vertices in combinations(graph.vertices, r):
        out_edges = graph.outgoing_edge_indices(vertices)
        subtour_constraints.append(sum(ye[out_edges]) >= 1)

In [ ]:
prob = graph.solve_from_ilp(ilp_constraints + subtour_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

# Solver callback

In [ ]:
def expand_cycle(graph, vertex, ye_values):
    cycle = [vertex]
    while True:
        out_edges = graph.outgoing_edge_indices(cycle[-1])
        y_out = ye_values[out_edges]
        k = out_edges[np.argmax(y_out)]
        next_vertex = graph.edges[k].head
        if next_vertex == cycle[0]:
            break
        cycle.append(next_vertex)
    return cycle

def shortest_subtour(graph, ye_values):
    vertices_left = graph.vertices
    subtour = graph.vertices
    while len(vertices_left) > 0:
        new_subtour = expand_cycle(graph, vertices_left[0], ye_values)
        if len(new_subtour) < len(subtour):
            subtour = new_subtour
        vertices_left = [v for v in vertices_left if v not in new_subtour]
    return subtour

def subtour_elimination(yv, ye):
    ye_value = np.array([y.value for y in ye])
    subtour = shortest_subtour(graph, ye_value)
    if len(subtour) == graph.num_vertices():
        return []
    else:
        print(f"Eliminated subtour with {len(subtour)} vertices.")
        out_edges = graph.outgoing_edge_indices(subtour)
        return [sum(ye[out_edges]) >= 1]

In [ ]:
prob = graph.solve_from_ilp(ilp_constraints, callback=subtour_elimination)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)